In [43]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dot
from tensorflow.keras.models import Model
from baseline_utils import create_labels_and_train

In [44]:
users_final = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/users_final_numeric.csv')
products_final= pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/products_final_numeric.csv')

In [45]:
# drop 'product_id' from both dataframe
products_final = products_final.drop(columns=['name', 'sku', 'id', 'name_embedding', 'name_embedding_flat'])
users_final = users_final.apply(lambda x: x.astype(float))
products_final = products_final.apply(lambda x: x.astype(float))

In [46]:
print(f"{users_final.shape}")
print(f"{products_final.shape}")

(80000, 17)
(19696, 33)


### Model 1: Deep Dense 
User tower = Dense128 + Dense 64
Product tower = Dense128 + Dense 64

In [47]:
user_input = Input(shape=(users_final.shape[1],), name='user_input')
item_input = Input(shape=(products_final.shape[1],), name='item_input')
user_tower = Dense(128, activation='relu')(user_input)
user_tower = Dense(64, activation='relu')(user_tower)
item_tower = Dense(128, activation='relu')(item_input)
item_tower = Dense(64, activation='relu')(item_tower)
dot_product = Dot(axes=1)([user_tower, item_tower])
model1 = Model(inputs=[user_input, item_input], outputs=dot_product)
model1.compile(optimizer='adam', loss='binary_crossentropy')
model1.summary()

Model: "functional_13"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 17)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ item_input          │ (None, 33)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_42 (Dense)    │ (None, 128)       │      2,304 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_44 (Dense)    │ (None, 128)       │      4,352 │ item_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 64)        │      8,256 │ dense_42[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_45 (Dense)    │ (None, 64)        │      8,256 │ dense_44[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_13 (Dot)        │ (None, 1)         │          0 │ dense_43[0][0],   │
│                     │                   │            │ dense_45[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 23,168 (90.50 KB)

 Trainable params: 23,168 (90.50 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
batch_size = 500
num_epochs = 25

create_labels_and_train(users_final, products_final, model1, batch_size, num_epochs)

/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14

### Model 2: Dense Medium
User tower = Dense128 
Product tower = Dense128

In [40]:
user_input = Input(shape=(users_final.shape[1],), name='user_input')
item_input = Input(shape=(products_final.shape[1],), name='item_input')
user_tower = Dense(128, activation='relu')(user_input)
item_tower = Dense(128, activation='relu')(item_input)
dot_product = Dot(axes=1)([user_tower, item_tower])
model2 = Model(inputs=[user_input, item_input], outputs=dot_product)
model2.compile(optimizer='adam', loss='binary_crossentropy')
#model2.summary()

create_labels_and_train(users_final, products_final, model2, batch_size, num_epochs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 511ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s